In [1]:
import pandas
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import time
import math

In [2]:
df_train = pandas.read_csv('./train.csv')
df_test = pandas.read_csv('./test.csv')
df_train = df_train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
df_test = df_test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)


In [3]:
x_train = torch.FloatTensor(math.floor(df_train.shape[0] * 0.9), df_train.shape[1] + 1)
y_train = torch.FloatTensor(math.floor(df_train.shape[0] * 0.9), 1)
x_valid = torch.FloatTensor(math.ceil(df_train.shape[0] * 0.1), df_train.shape[1] + 1)
y_valid = torch.FloatTensor(math.ceil(df_train.shape[0] * 0.1), 1)
x_test = torch.FloatTensor(df_test.shape[0], df_test.shape[1] + 2)

print(x_train.shape, x_test.shape, x_valid.shape, y_valid.shape, y_train.shape)

age_mean = np.mean(df_train['Age'])
print(age_mean)

for i in range(x_train.shape[0]):
    y_train[i][0] = df_train['Survived'][i]
    x_train[i][0] = df_train['Pclass'][i]
    x_train[i][1] = 0 if df_train['Sex'][i] == 'male' else 1
    x_train[i][2] = age_mean if pandas.isnull(df_train['Age'][i]) else df_train['Age'][i] // 1
    x_train[i][3] = df_train['SibSp'][i]
    x_train[i][4] = df_train['Parch'][i]
    x_train[i][5] = df_train['Fare'][i] 
    x_train[i][6] = 1 if df_train['Embarked'][i] == 'S' else 0
    x_train[i][7] = 1 if df_train['Embarked'][i] == 'C' else 0
    x_train[i][8] = 1 if df_train['Embarked'][i] == 'Q' else 0

for i in range(x_valid.shape[0]):
    y_valid[i][0] = df_train['Survived'][i + x_train.shape[0]]
    x_valid[i][0] = df_train['Pclass'][i + x_train.shape[0]]
    x_valid[i][1] = 0 if df_train['Sex'][i + x_train.shape[0]] == 'male' else 1
    x_valid[i][2] = age_mean if pandas.isnull(df_train['Age'][i + x_train.shape[0]]) else df_train['Age'][i + x_train.shape[0]] // 1
    x_valid[i][3] = df_train['SibSp'][i + x_train.shape[0]]
    x_valid[i][4] = df_train['Parch'][i + x_train.shape[0]]
    x_valid[i][5] = df_train['Fare'][i + x_train.shape[0]]
    x_valid[i][6] = 1 if df_train['Embarked'][i + x_train.shape[0]] == 'S' else 0
    x_valid[i][7] = 1 if df_train['Embarked'][i + x_train.shape[0]] == 'C' else 0
    x_valid[i][8] = 1 if df_train['Embarked'][i + x_train.shape[0]] == 'Q' else 0

for i in range(x_test.shape[0]):
    x_test[i][0] = df_test['Pclass'][i]
    x_test[i][1] = 0 if df_test['Sex'][i] == 'male' else 1
    x_test[i][2] = age_mean if pandas.isnull(df_test['Age'][i]) else df_test['Age'][i] // 1
    x_test[i][3] = df_test['SibSp'][i]
    x_test[i][4] = df_test['Parch'][i]
    x_test[i][5] = df_test['Fare'][i]



torch.Size([801, 6]) torch.Size([418, 6]) torch.Size([90, 6]) torch.Size([90, 1]) torch.Size([801, 1])
29.69911764705882


In [4]:
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x_data = x
        self.y_data = y

    def __len__(self):
        return len(self.y_data)

    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])

        return x, y

dataset_train = CustomDataset(x_train, y_train)
dataset_valid = CustomDataset(x_valid, y_valid)

print(x_valid)

batch_size = 8
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
dataloader_valid = DataLoader(dataset_valid, batch_size=batch_size, shuffle=True)

tensor([[  2.0000,   1.0000,  31.0000,   1.0000,   1.0000,  26.2500],
        [  1.0000,   0.0000,  11.0000,   1.0000,   2.0000, 120.0000],
        [  3.0000,   0.0000,   0.0000,   0.0000,   1.0000,   8.5167],
        [  3.0000,   0.0000,  27.0000,   0.0000,   0.0000,   6.9750],
        [  3.0000,   0.0000,  31.0000,   0.0000,   0.0000,   7.7750],
        [  1.0000,   0.0000,  39.0000,   0.0000,   0.0000,   0.0000],
        [  3.0000,   1.0000,  18.0000,   0.0000,   0.0000,   7.7750],
        [  2.0000,   0.0000,  39.0000,   0.0000,   0.0000,  13.0000],
        [  1.0000,   1.0000,  33.0000,   1.0000,   0.0000,  53.1000],
        [  3.0000,   0.0000,  26.0000,   0.0000,   0.0000,   7.8875],
        [  3.0000,   0.0000,  39.0000,   0.0000,   0.0000,  24.1500],
        [  2.0000,   0.0000,  35.0000,   0.0000,   0.0000,  10.5000],
        [  3.0000,   1.0000,   6.0000,   4.0000,   2.0000,  31.2750],
        [  3.0000,   0.0000,  30.0000,   0.0000,   0.0000,   8.0500],
        [  1.0000,  

In [5]:
class net1(nn.Module):
    def __init__(self):
        super(net1, self).__init__()
        self.net = nn.Sequential(nn.Linear(6, 16), nn.ReLU(), nn.Linear(16, 16), nn.ReLU(),
        nn.Linear(16, 16), nn.ReLU(), nn.Linear(16, 1), nn.Sigmoid())

    def forward(self, x):
        y = self.net(x)
        return y

In [6]:
device = 'cpu'

def train(net, trainloader, validloader, num_epoch, optimizer, model_path): # Function to train the network
    for epoch in range(num_epoch):
        start_time = time.time()
        running_loss = 0.0

        for i, data in enumerate(trainloader, 0):
            x, y = data
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            y_pred = net(x)
            loss = F.binary_cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # print training stats
        average_loss = running_loss / len(trainloader)
        with torch.no_grad():
            validation_loss, validation_acc = val(net, epoch, validloader)
        end_time = time.time()
        elapsed_time = end_time - start_time
            
        print('Epoch: %d / Training Loss: %.3f / Validation Loss: %.3f / Time : %.2f (s) / acc : %.3f' %
              (epoch+1, average_loss, validation_loss, elapsed_time, validation_acc))


    print('Finished Training')
    torch.save(net, model_path)
    print('Saved Trained Model')

def val(net, current_epoch, validloader): # Function to validate the network
    net.eval()
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(validloader, 0):
        x, y = data
        x = x.to(device)
        y = y.to(device)
        y_pred = net(x)
        running_acc += np.sum(np.where(abs(y_pred - y) < 0.5, 1, 0))
        loss = F.binary_cross_entropy(y_pred, y)
        running_loss += loss.item()

    average_loss = running_loss / len(validloader)
    average_acc = running_acc / len(validloader) / batch_size
    net.train()

    return average_loss, average_acc

In [7]:
epoch = 30
lr = 0.001
beta1 = 0.5
beta2 = 0.999
reg = 0
net = net1()

model_path = './model1.pth'

for i in range(8):
    model_path = './model' + str(i) + '.pth'
    optimizer = torch.optim.Adam(net.parameters(),lr=lr, betas=(beta1, beta2), weight_decay=reg)
    train(net, dataloader_train, dataloader_valid, epoch, optimizer, model_path)

Epoch: 1 / Training Loss: 0.659 / Validation Loss: 0.598 / Time : 0.08 (s) / acc : 0.635
Epoch: 2 / Training Loss: 0.615 / Validation Loss: 0.571 / Time : 0.05 (s) / acc : 0.656
Epoch: 3 / Training Loss: 0.600 / Validation Loss: 0.534 / Time : 0.05 (s) / acc : 0.667
Epoch: 4 / Training Loss: 0.589 / Validation Loss: 0.530 / Time : 0.05 (s) / acc : 0.656
Epoch: 5 / Training Loss: 0.579 / Validation Loss: 0.521 / Time : 0.05 (s) / acc : 0.698
Epoch: 6 / Training Loss: 0.577 / Validation Loss: 0.516 / Time : 0.05 (s) / acc : 0.719
Epoch: 7 / Training Loss: 0.558 / Validation Loss: 0.500 / Time : 0.07 (s) / acc : 0.750
Epoch: 8 / Training Loss: 0.536 / Validation Loss: 0.431 / Time : 0.05 (s) / acc : 0.750
Epoch: 9 / Training Loss: 0.526 / Validation Loss: 0.429 / Time : 0.05 (s) / acc : 0.750
Epoch: 10 / Training Loss: 0.508 / Validation Loss: 0.419 / Time : 0.05 (s) / acc : 0.750
Epoch: 11 / Training Loss: 0.489 / Validation Loss: 0.418 / Time : 0.05 (s) / acc : 0.771
Epoch: 12 / Trainin

In [8]:
y_test = torch.zeros((df_test.shape[0], 1))

for i in range(8):
    net = torch.load('./model' + str(i) + '.pth')
    net.eval()
    y_test += net(x_test)


y_test = np.where(y_test < 4, 0, 1)
y_test = np.squeeze(y_test)

In [9]:
result = dict()
result['PassengerId'] = [i + 892 for i in range(y_test.shape[0])]
result['Survived'] = y_test
result = pandas.DataFrame(result)

print(result)
result.to_csv('./submission.csv')

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
